# Kulo

In [1]:
print("kulo pt. 2")

kulo pt. 2


## 1. Import geojson (parsing .geojson files)

In [2]:
import geojson

## 2. Create load_data function to load geojson data

In [3]:
def load_data(filename):
    """
    Loads GeoJson Data from "filename"
    """
    with open(filename) as f:
        data = geojson.load(f)
    return data


## 3. load data from WA DNR 1970-2007 fire statistics

In [4]:
older_fire_data = load_data("data/DNR_Fire_Statistics_1970-2007.geojson")

## 3.5. load data from WA DNR 2008-present fire statistics

In [5]:
newer_fire_data = load_data("data/DNR_Fire_Statistics_1970-2007.geojson")

## 4. pull out "features" section of data (that's where fire data is)

In [6]:
old_fire_data = older_fire_data["features"]
new_fire_data = newer_fire_data["features"]

## 5. make sure both datasets are imported by checking total number of fires 

In [7]:
# get total number of fires
old_total = len(old_fire_data)
new_total = len(new_fire_data)
print(old_total + new_total)

76232


## 6. make clean dataset from both pieces of data (only data-points we need)

In [8]:
cleaned_fire_data = []
for fire in old_fire_data:
    date = fire["properties"]["START_DT"]
    acres = fire["properties"]["ACRES_BURNED"]
    lon = fire["geometry"]["coordinates"][0]
    lat = fire["geometry"]["coordinates"][1]
    cleaned_fire_data.append((date, acres, lat, lon))

for fire in new_fire_data: 
    date = fire["properties"]["START_DT"]
    acres = fire["properties"]["ACRES_BURNED"]
    lon = fire["geometry"]["coordinates"][0]
    lat = fire["geometry"]["coordinates"][1]
    cleaned_fire_data.append((date, acres, lat, lon))

In [10]:
print(len(cleaned_fire_data))

76232


## 7. import csv (comma-separated values)

In [11]:
import csv

## 8. save cleaned data to csv

In [15]:
with open('data/clean_fire_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile,)
    writer.writerow(("date", "acres", "lat", "lon"))
    for fire in cleaned_fire_data:
        writer.writerow(fire)